In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm,inv
%matplotlib inline

In [2]:
A,M,lamda = 4,81,0.9925
target = [3,11,12,15,16,17,20,22,23,24,26,29,30,31,34,35
          ,39,43,48,52,53,56,57,58,59,60,61,62,66,70,71]
target = np.array(target) - 1

In [3]:
def reconstruct(sparse):
    result = np.zeros(M**2)
    index = ((sparse[:,0] - 1) * M + sparse[:,1]).astype(int)
    result[index-1] = sparse[:,2]
    return result

In [4]:
# read data from txt files
prob_a1_sparse = np.loadtxt('prob_a1.txt')
prob_a2_sparse = np.loadtxt('prob_a2.txt')
prob_a3_sparse = np.loadtxt('prob_a3.txt')
prob_a4_sparse = np.loadtxt('prob_a4.txt')
R = np.loadtxt('rewards.txt')

In [5]:
# reconstruct the transition matrix
prob_a1 = reconstruct(prob_a1_sparse).reshape(M,M)
prob_a2 = reconstruct(prob_a2_sparse).reshape(M,M)
prob_a3 = reconstruct(prob_a3_sparse).reshape(M,M)
prob_a4 = reconstruct(prob_a4_sparse).reshape(M,M)
prob = np.array([prob_a1,prob_a2,prob_a3,prob_a4])
print(prob.shape)

(4, 81, 81)


### a) policy iteration:

In [6]:
# initialize policy pi randomly
pi = np.random.randint(4,size = 81)

In [7]:
def value(P_pi,R):
    return (inv(np.eye(M)-lamda*P_pi)).dot(R)

In [8]:
def choose(pi,prob):
    result = np.zeros((M,M))
    for i in range(M):
        result[i,:] += prob[pi[i]][i,:]
    return result

In [9]:
def greedy(V_pi):
    temp = (prob.dot(V_pi[:,np.newaxis])).reshape(4,81).T
    return np.argmax(temp,axis = 1)

In [10]:
pi_old = np.zeros(81)
iteration_p = 0
while (np.array_equal(pi_old,pi) == False):
    iteration_p += 1
    pi_old = pi
    P_pi = choose(pi,prob)
    V_pi = value(P_pi,R)
    pi = greedy(V_pi)
    if iteration_p >= 100:
        break

In [11]:
policy_iteration = pi[target]
print(policy_iteration.shape)

(31,)


### b) value iteration:

In [12]:
# initialize values to be zero, policy to be -1(invalid value)
V_old = np.zeros(M) - 1
V = np.zeros(M)

In [15]:
iteration_v = 0
while (norm(V_old-V) >= 0.005):
    iteration_v += 1
    V_old = V
    temp = (prob.dot(V_old[:,np.newaxis])).reshape(4,81).T
    V = np.max(temp,axis = 1) * lamda + R
    if iteration_v >= 1000:
        break

In [16]:
pi = np.argmax(temp,axis = 1)

In [17]:
pi[target]

array([2, 2, 1, 3, 3, 2, 2, 3, 3, 0, 2, 3, 3, 0, 2, 1, 0, 2, 0, 2, 2, 3,
       3, 3, 3, 3, 2, 2, 0, 2, 1])

In [19]:
print(policy_iteration - pi[target])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
